# DARKS - Nico Keeghan

We start by importing all the libraries we are interested in.

In [1]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, combiner
from astropy import units as u
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

Load in the darks. We had 6 from the night that we oberved (28/03). We also decided to use the darks from 25/03, of which there are 24. Since some of the images included light leaks, we will only use 13 images (2857-2869) from 25/03, and 2 images (3661, 3662) from 28/03.

In [13]:
directory = "darks"
images = ccdproc.ImageFileCollection("./" + directory, glob_include = 'Dark_*')    # loads in all of the darks

In [15]:
# make sure the files have been loaded
print(len(images.files))

15


There are 15 files, as expected.

Load in all of the dark filenames.

In [16]:
# loops through the filenames and reads the data for each file
darks = [ CCDData.read(directory + '/' + fn, unit = "adu") for fn in images.files_filtered(PICTTYPE = 3) ]

We check that all of the darks have been included.

In [17]:
len(darks)

15

All 15 files have been loaded!

We start by checking some statistics of the first dark.

In [18]:
image = darks[0]

print('Exposure time:',image.header['EXPTIME'], 'seconds')

print('Min:', np.min(image))
print('Max:', np.max(image))
print('Median:', np.median(image))
print('Std Dev:', np.std(image))

Exposure time: 3600.0 seconds
Min: 2102
Max: 65535
Median: 2297.0
Std Dev: 676.6414066850311


The median of the bias image was 2079 counts. The mean of this dark image is slightly larger than that, which is expected as the dark will contain all of the electronics and detector noise as well as contributions from the dark current. The standard deviation is very large, which is due to the broad range of values - most pixels will have counts a little larger than the median value of the bias, but some hot pixels will have very high counts. Given that the maximum counts detected is 65535, some pixels are likely to be saturated due to dark current.

We now need to subtract the median-combined bias from the darks.

In [19]:
bias_median = CCDData.read('bias_median.fits', unit = 'adu')

for idx, thisimage in enumerate(darks): 
    darks[idx] = ccdproc.subtract_bias(thisimage, bias_median)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


We will examine the first image with the bias subtracted.

In [20]:
print('Min:', np.min(darks[0]))
print('Max:', np.max(darks[0]))
print('Median:', np.median(darks[0]))
print('Std Dev:', np.std(darks[0]))

Min: 26.0
Max: 63459.5
Median: 215.5
Std Dev: 676.3931573650432


These statistics look good - the median is 215.5 counts, which is roughly the difference between the counts of the image and the counts of the median combined bias image. The standard deviation is still large because we have just subtracted off the bias - there are still a lot of hot pixels that cause this large standard deviation.

In [21]:
dark_median = ccdproc.Combiner(darks, dtype=np.float32).median_combine() # median combine the dark images

In [22]:
print('Statistics for median-combined dark image')

print('Min:', np.min(dark_median))
print('Max:', np.max(dark_median))
print('Median:', np.median(dark_median))
print('Std Dev:', np.std(dark_median))



Statistics for median-combined dark image
Min: 35.0
Max: 63459.5
Median: 193.0
Std Dev: 671.94189453125


Again, these statitics look reasonable. The median of 193.0 counts for a 3600 second exposure seems reasonable. The standard deviation is still large because the hot pixels are in the same location in each image, so they remain in the combined image.

In [23]:
fig, ax = plt.subplots(figsize = (10,10))
plt.imshow(dark_median, cmap = 'Greys', norm = LogNorm())
plt.title('Median dark with bias subtracted')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.show()

***Figure 1.*** Median dark image with bias subtracted. It appears uniform with a few bright hot pixels.

This dark looks good - it is very uniform except for the hot pixels, which are much brighter than the other pixels. We can't see any obvious structure in the image, except for the top left corner, which is slightly brighter. This bright corner is consistent in all of the dark images.

In [24]:
dark_median.meta.update(EXPTIME = 3600)           # add exposure time to dark_median header
dark_median.meta.update(TELESCOP = 'C14')         # add telescope used
dark_median.meta.update(OBJECT = 'Dark_Median')   # add object name so we can identify the dark median image
dark_median.write("dark_median.fits")             # write the dark_median image to a fits file

In [25]:
del(darks)
collected = gc.collect()
print('Check garbage collection', collected)

Check garbage collection 2278
